<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Mohammad Javad Maheronnaghsh
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [4]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-05 16:49:53--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-05 16:49:53--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce6e8ee17be2e8a20f1fe0da056.dl.dropboxusercontent.com/cd/0/get/B7cJUR9dWjbEN9XSGd2chUSAXdouCK7EzjTg1YZA4DapETdBNjrBaXxpm47yUT1PiQKSSElaAK7YZMBA_NSnupWm1Wgxprb34SjXHU2uf-IjQ26eow_wkVT0MkrfC-i4_4N3aw4zRp2CsXWD-zFaTp_ScOkUVcbhkw04AFfEiPnHw4WaiF84Xkmh0rqn8FXa95k/file?dl=1# [following]
--2023-05-05 16:49:53--  https://uce6e8ee17be2e8a20f1fe0da056.dl.dropboxusercontent.com/cd/0/get/B7cJUR9dWjbEN9XSGd2chUSAXdouCK7EzjTg1YZA4D

In [5]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

Archive:  /content/dataset.zip
  inflating: /content/dataset/content/news-train.csv  


# Preprocessing

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [7]:
df = pd.read_csv('/content/dataset/content/news-train.csv')

In [8]:
df

,topic,text
0,sports,به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1,politics,پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2,politics,# دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3,economics,کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4,economics,به گزارش ایسنا صحبت با برخی # # آن است که امرو...
...,...,...
197356,sports,به گزارش ورزش سه و به نقل از مارکا رقابت‌های ب...
197357,sports,به گزارش ایسنا و به نقل از فوتبال ایتالیا یوون...
197358,sports,سرمربی تیم والیبال دانشجویان گفت ما اصلا فرصت ...
197359,sports,به گزارش ورزش سه مرتضی # باشگاه # مصر # را نسب...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [9]:
occurancies = df['topic'].value_counts()
occurancies

sports       123105
economics     49600
cultural      13359
politics      11297
Name: topic, dtype: int64

Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [10]:
# Find the topic with the minimum occurancy
min_news_arg = np.argmin(occurancies)
list_of_topics = occurancies.index.tolist()
min_news = list_of_topics[min_news_arg]
min_occurancy = occurancies[min_news_arg]
print('The topic with the lowest number of occurancies is : ' + min_news + 
      ' and the occurancy of this topic is : ' + str(min_occurancy))



The topic with the lowest number of occurancies is : politics and the occurancy of this topic is : 11297


In [11]:

# Downsampling
new_df = pd.DataFrame({'topic':[], 'text':[]})
for other_topic in list_of_topics:
  if other_topic != min_news:
    downsampled = np.random.choice(df[df.topic == other_topic]['text'], size=min_occurancy, replace=False)
    topic = np.array([other_topic for i in range(min_occurancy)]).reshape((min_occurancy,))
    print(topic.shape)
    
    downsampled = np.stack([topic, downsampled])
    new_df = new_df.append(pd.DataFrame(downsampled.T, columns = ['topic', 'text']))
    print(new_df.shape)
    type(downsampled)
  else:
    not_downsampled = np.array(df[df.topic == min_news]['text'])
    topic = np.array([min_news for i in range(min_occurancy)]).reshape((min_occurancy,))
    not_downsampled = np.stack([topic, not_downsampled])
    new_df = new_df.append(pd.DataFrame(not_downsampled.T, columns = ['topic', 'text']))


(11297,)
(11297, 2)
(11297,)
(22594, 2)
(11297,)
(33891, 2)


<ipython-input-11-ab1bcfa79da1>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.DataFrame(downsampled.T, columns = ['topic', 'text']))
<ipython-input-11-ab1bcfa79da1>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.DataFrame(downsampled.T, columns = ['topic', 'text']))
<ipython-input-11-ab1bcfa79da1>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.DataFrame(downsampled.T, columns = ['topic', 'text']))
<ipython-input-11-ab1bcfa79da1>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.DataFrame(not_downsampled.T, columns = ['topic'

In [12]:
new_df

,topic,text
0,sports,# سواری در خراسان شمالی روزهای # را می‌گذارند ...
1,sports,به گزارش ورزش سه لیورپول فردا شب در شرایطی در ...
2,sports,به گزارش ایسنا # # سرمربی تیم ملی والیبال بعد ...
3,sports,به گزارش ورزش سه در هفته بیست و ششم رقابت‌های ...
4,sports,به گزارش ورزش سه با اینکه فصل سرد فرا رسیده و ...
...,...,...
11292,politics,آیین # و # فرمانده نیروی دریایی سپاه دیروز با ...
11293,politics,رئیس‌جمهور با اشاره به تلاش‌های انجام شده برای...
11294,politics,عضو مجمع تشخیص مصلحت نظام گفت در حال حاضر حاکم...
11295,politics,پس از پایان دور ششم مذاکرات وین طرفین به مثبت ...


# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [13]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(new_df, test_size=0.2)

In [14]:
print('Size of train is : ' + str(round(len(df_train)/ (len(df_train) + len(df_test)), 2)))

Size of train is : 0.8


In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [15]:
df_train

,topic,text
8547,cultural,وزیر فرهنگ و ارشاد اسلامی صبح دیروز در دیدار ب...
4183,cultural,ارکستر ملی ایران به رهبری فریدون # در حالی به ...
1131,economics,مخالفت بانک مرکزی به عنوان اصلی‌ترین نهاد تصمی...
6254,politics,در فرایند برگزاری انتخابات همکاری مناسبی بین د...
7682,politics,اعتماد نگران ناامید خسته عباس عبدی نگران ناامی...
...,...,...
3261,cultural,در حال حاضر وضع ادبیات معاصر در دانشگاه‌ها به ...
9862,sports,به‌ گزارش ‌ایسنا رقابت‌های سه وزن دوم و پایانی...
6382,economics,مشاور وزیر تعاون کار و رفاه اجتماعی در امور با...
6064,sports,به گزارش ورزش سه # که در اولین بازی شان در شرو...


In [16]:
# !pip install hazm

In [17]:
# from __future__ import unicode_literals
# from hazm import *
# # test
# word_tokenize ( 'ولی برای پردازش، جدا بهتر نیست؟' )

In [18]:
all_tokens_list = []
all_tokens_flattened = []
for index, row in df_train.iterrows():
  tokenized = row['text'].split()
  for token in tokenized:
    all_tokens_flattened.append(token)
  all_tokens_list.append(tokenized)
# print(all_tokens_list)
# TODO: SPLIT BASED ON space instead of 'hazm'

In [19]:
len(all_tokens_list)

36150

Count the tokens and list the top 5 frequent ones (5 points)

In [20]:
print(len(all_tokens_flattened))

19540886


In [21]:
flattened_tokens = pd.DataFrame(all_tokens_flattened)
top_5 = flattened_tokens.value_counts()[:5]
top_5

#     1631961
و      881712
در     746708
به     585050
از     437573
dtype: int64

Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [22]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-05 16:50:21--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-05 16:50:21--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-05 16:50:21 (40.3 M

In [23]:
f = open('/content/stopwords.txt', 'r')
stop_words = f.read()
print(len(stop_words))

1637


In [24]:
# Delete the stop words in the list of articles and their tokens
all_tokens_removed_stop_words = []
from time import sleep
from progressbar import progressbar

for i in progressbar(range(len(all_tokens_list))):
  list_ = all_tokens_list[i]
  new_list = []
  for token in list_:
    if token not in stop_words and token != '#':
      new_list.append(token)
  all_tokens_removed_stop_words.append(new_list)

100% (36150 of 36150) |##################| Elapsed Time: 0:00:19 Time:  0:00:19


In [25]:
print(len(all_tokens_removed_stop_words))

36150


In [26]:
# Delete the stop words in the list of all tokens
all_tokens_flattened_removed_stop_words = []
for token in all_tokens_flattened:
  if token not in stop_words and token != '#':
    all_tokens_flattened_removed_stop_words.append(token)
  

In [27]:
print(len(all_tokens_flattened_removed_stop_words))

10706615


In [28]:
# Free space
del(all_tokens_flattened)
del(all_tokens_list)

Make a list of the remaining unique tokens (5 points)

In [29]:
unique_tkns = set(all_tokens_flattened_removed_stop_words)
print(len(unique_tkns))

4717


Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [30]:
print(len(df_train))

36150


In [31]:
token_dict = dict()
set_to_list = list(unique_tkns)
for token_idx in range(len(unique_tkns)):
  token = set_to_list[token_idx]
  token_dict[token] = token_idx

In [32]:
token_dict

{'گواردیولا': 0,
 'متاسفانه': 1,
 'زدند': 2,
 'منتقدان': 3,
 'پیمانکاران': 4,
 'سرخیو': 5,
 'کین': 6,
 'کوچک': 7,
 'پیامبر': 8,
 'دچار': 9,
 'دیوار': 10,
 'نگرش': 11,
 'زنی': 12,
 'گل\u200cمحمدی': 13,
 'می\u200cگیرد': 14,
 'پنجشنبه': 15,
 'قاسم': 16,
 'دوست': 17,
 'عبارت': 18,
 'درمانی': 19,
 'جزئیات': 20,
 'خانم': 21,
 'طبیعی': 22,
 'آوردیم': 23,
 'اردوی': 24,
 'مصاحبه\u200cای': 25,
 'دادم': 26,
 'آتش': 27,
 'تاسیس': 28,
 'متفاوتی': 29,
 'آنقدر': 30,
 'لباس': 31,
 'ارجاع': 32,
 'نصب': 33,
 'نعمتی': 34,
 'اجتماعی': 35,
 'توییتر': 36,
 'زیبا': 37,
 'نویسندگی': 38,
 'راموس': 39,
 'رهبر': 40,
 'واقعه': 41,
 'تقدیر': 42,
 'جنجالی': 43,
 'فعلی': 44,
 'دایره': 45,
 'پشتوانه': 46,
 'تبدیل': 47,
 'گام\u200cهای': 48,
 'حالت': 49,
 'مسعود': 50,
 'عزم': 51,
 'مانع': 52,
 'وظایف': 53,
 'تدریج': 54,
 'می\u200cخواهد': 55,
 'مشکلی': 56,
 'سارا': 57,
 'شخصیت': 58,
 'مارادونا': 59,
 'روستایی': 60,
 'شمس': 61,
 'تلاش': 62,
 'پست': 63,
 'کوتاهی': 64,
 '\u200c': 65,
 'امتیاز': 66,
 'تعویق': 67,
 'می\u200c

In [33]:

doc_representations = []
from time import sleep
from progressbar import progressbar

for doc_num in progressbar(range(len(df_train))):
  current_doc = [0 for i in range(len(unique_tkns))]
  for token in all_tokens_removed_stop_words[doc_num]:
    # occurancies = all_tokens_removed_stop_words[doc_num].count(token)
    # current_doc.append(occurancies)
    token_idx = token_dict[token]
    current_doc[token_idx] += 1
    
  doc_representations.append(current_doc)
    
    # Maybe we can increase the speed by looping over all tokens and add 

100% (36150 of 36150) |##################| Elapsed Time: 0:00:10 Time:  0:00:10


In [34]:
print(doc_representations[2])
print(len(doc_representations))
print(len(doc_representations[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [35]:
x_train = np.array(doc_representations)

In [36]:
print(x_train.shape)

(36150, 4717)


Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [37]:
y_train = np.array([])
for topic in df_train['topic']:
  if topic == 'sports':
    y_train = np.append(y_train, 3)
  elif topic == 'economics':
    y_train = np.append(y_train, 2)
  elif topic == 'politics':
    y_train = np.append(y_train, 1)
  elif topic == 'cultural':
    y_train = np.append(y_train, 4)
  else:
    print('Error')

In [38]:
y_train.shape

(36150,)

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [39]:
tkns_list = []
for index, row in df_test.iterrows():
  tokenized = row['text'].split()
  tkns_list.append(tokenized)

tkns_removed_stps_test = []

for i in progressbar(range(len(tkns_list))):
  list_ = tkns_list[i]
  new_list = []
  for token in list_:
    if token not in stop_words and token != '#' and token in unique_tkns:
      new_list.append(token)
  tkns_removed_stps_test.append(new_list)

100% (9038 of 9038) |####################| Elapsed Time: 0:00:05 Time:  0:00:05


In [40]:
dec_test = []
for doc_num in progressbar(range(len(df_test))):
  current_doc = [0 for i in range(len(unique_tkns))]
  for token in tkns_removed_stps_test[doc_num]:
    token_idx = token_dict[token]
    current_doc[token_idx] += 1
    
  dec_test.append(current_doc)

100% (9038 of 9038) |####################| Elapsed Time: 0:00:02 Time:  0:00:02


In [41]:
print(len(dec_test[0]))
print(len(dec_test))
x_test = dec_test

4717
9038


Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [42]:
y_test = np.array([])
for topic in df_test['topic']:
  if topic == 'sports':
    y_test = np.append(y_test, 3)
  elif topic == 'economics':
    y_test = np.append(y_test, 2)
  elif topic == 'politics':
    y_test = np.append(y_test, 1)
  elif topic == 'cultural':
    y_test = np.append(y_test, 4)
  else:
    print('Error')

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [43]:
print(x_train.shape)
print(y_train.shape)

(36150, 4717)
(36150,)


In [44]:
# This is its documentation: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(force_alpha=True)
clf.fit(x_train, y_train)


MultinomialNB(force_alpha=True)

Get the predictions of the model for the testing news (`x_test`).

In [45]:
predicts_sklearn_mlt = clf.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [46]:
# documentation: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
from sklearn.metrics import classification_report
print(classification_report(y_test, predicts_sklearn_mlt, target_names=['0', '1', '2', '3']))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      2229
           1       0.91      0.94      0.93      2269
           2       0.99      0.98      0.99      2242
           3       0.94      0.96      0.95      2298

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [99]:
class Naive_Bayes:
    def __init__(self):
        """
        YOUR CODE (IF NECESSARY)
        """
  
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        """
        YOUR CODE
        """
        self.all_classes = np.unique(y)
        n_classes = len(self.all_classes)

        self.priors = np.zeros(n_classes)
        self.likelihoods = np.zeros((n_classes, n_features))

        for index, class_ in enumerate(self.all_classes):
            X_ = X[class_ == y]
            self.priors[index] = X_.shape[0] / n_samples
            self.likelihoods[index, :] = ((X_.sum(axis=0)) + 1) / (np.sum(X_.sum(axis=0) + 1))
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        y_pred = []
        for x in X:
          posteriors = []
          for index, _ in enumerate(self.all_classes):
              prior = np.log(self.priors[index])
              likelihoods_ = np.log(self.likelihoods[index,:]) * x
              posteriors.append(np.sum(likelihoods_) + prior)
          y_pred.append(self.all_classes[np.argmax(posteriors)])
        return y_pred

In [50]:
y_train.shape

(36150,)

Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [100]:
my_own_model = Naive_Bayes()
my_own_model.fit(x_train, y_train)

Get the predictions of your model for the testing news (`x_test`).

In [101]:
predicts_own_model = my_own_model.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [102]:
print(classification_report(y_test, predicts_own_model, target_names=['0', '1', '2', '3']))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      2229
           1       0.91      0.94      0.93      2269
           2       0.99      0.98      0.99      2242
           3       0.94      0.96      0.95      2298

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef

In [ ]:
'''
here we are going to predict the test files
given in kaggle
we will predict based on the pre-constructed model
in this assignment (as it was before, in the previous HWs)
'''
# I have uploaded the test file to my google drive
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
# read csv
!touch "/content/gdrive/My Drive/assignment4-test-data.csv"
test_file_df = pd.read_csv('/content/gdrive/My Drive/assignment4-test-data.csv')

In [ ]:
test_file_df

In [ ]:
tkns_list_kaggle = []
for index, row in test_file_df.iterrows():
  tokenized = row['text'].split()
  tkns_list_kaggle.append(tokenized)

tkns_removed_stps_test_kaggle = []

for i in progressbar(range(len(tkns_list_kaggle))):
  list_ = tkns_list_kaggle[i]
  new_list = []
  for token in list_:
    if token not in stop_words and token != '#' and token in unique_tkns:
      new_list.append(token)
  tkns_removed_stps_test_kaggle.append(new_list)

In [ ]:
dec_test_kaggle = []
for doc_num in progressbar(range(len(test_file_df))):
  current_doc = [0 for i in range(len(unique_tkns))]
  for token in tkns_removed_stps_test_kaggle[doc_num]:
    token_idx = token_dict[token]
    current_doc[token_idx] += 1
    
  dec_test_kaggle.append(current_doc)

In [ ]:
print(len(dec_test_kaggle[0]))
print(len(dec_test_kaggle))
x_test_kaggle = dec_test_kaggle

In [ ]:
predicts_sklearn_mlt = clf.predict(x_test_kaggle)
predicts_sklearn_mlt = np.array([int(x) for x in predicts_sklearn_mlt])

In [ ]:
topic = predicts_sklearn_mlt
ID = np.array([i+1 for i in range(500)])

In [ ]:
from google.colab import files
nparray = np.vstack([topic, ID])
df_to_download = pd.DataFrame(nparray.T, columns = ['topic', 'ID'])
from numpy import savetxt
savetxt('results.csv', df_to_download, delimiter=',')
files.download('results.csv')